> This is a preliminary study on shader augmentation

In [2]:
import os, sys
sys.path.append(os.path.join(os.path.abspath(''), '../'))

import peewee as pw
from toyDb.databases import ExperimentDb, ShaderDb
from toyDb.utils.Directory import getToyDbRootDir

import matplotlib.pyplot as plt
import numpy as np
import json
from tqdm import tqdm
import torch

ExperimentDb.init_from_default_db()

## Prepare

In [3]:
shaderDB = ShaderDb.ShaderDB(os.path.join(getToyDbRootDir(), "shaders"))
shaderDB.scan_local()
shaderDB.load_all()

all_shaders = set(shaderDB.offlineShaders.keys())
imageonly_shaders = shaderDB.filter_attribute(["is_imageonly"])

# all environments available in database
all_environments = ExperimentDb.Environment.select()

# shaders that have at least 1 non-error measurement
measurable_shaders = set([shdr.shader_id for shdr in ExperimentDb.ImageOnlyShader.select(
      ExperimentDb.ImageOnlyShader.shader_id
    ).join(ExperimentDb.ImageOnlyExperiment).where(
      ExperimentDb.ImageOnlyExperiment.errors == ExperimentDb.ErrorType.NONE,
    ).distinct()
])

# shaders that have measurements in all working environments
all_measurable_shaders = set([shdr.shader_id for shdr in ExperimentDb.ImageOnlyShader.select(
      ExperimentDb.ImageOnlyShader.shader_id
    ).join(ExperimentDb.ImageOnlyExperiment).group_by(
      ExperimentDb.ImageOnlyShader.shader_id
    ).having(
      pw.fn.MAX(ExperimentDb.ImageOnlyExperiment.errors) == 0
    )
])

measurable_and_traceable_shaders = set([shdr.shader_id for shdr in ExperimentDb.ImageOnlyShader.select(
      ExperimentDb.ImageOnlyShader.shader_id
    ).join(ExperimentDb.ImageOnlyExperiment).where(
      ExperimentDb.ImageOnlyExperiment.errors == ExperimentDb.ErrorType.NONE,
      ExperimentDb.ImageOnlyExperiment.trace.is_null(False)
    ).distinct()
])

all_measurable_and_traceable_shaders = set([shdr.shader_id for shdr in ExperimentDb.ImageOnlyShader.select(
      ExperimentDb.ImageOnlyShader.shader_id
    ).join(ExperimentDb.ImageOnlyExperiment).group_by(
      ExperimentDb.ImageOnlyShader.shader_id
    ).having(
      pw.fn.MAX(ExperimentDb.ImageOnlyExperiment.errors) == 0,
      pw.fn.SUM(ExperimentDb.ImageOnlyExperiment.trace.is_null(True)) == 0
    )
])

all_measurable_and_traceable_canonical_shaders = set([shdr.shader_id for shdr in ExperimentDb.ImageOnlyShader.select(
      ExperimentDb.ImageOnlyShader.shader_id
    ).join(ExperimentDb.ImageOnlyExperiment).where(
      ExperimentDb.ImageOnlyExperiment.num_cycles == ExperimentDb.CANONICAL_NUM_CYCLES,
      ExperimentDb.ImageOnlyExperiment.num_trials == ExperimentDb.CANONICAL_NUM_TRIALS,
      ExperimentDb.ImageOnlyExperiment.width == ExperimentDb.CANONICAL_WIDTH,
      ExperimentDb.ImageOnlyExperiment.height == ExperimentDb.CANONICAL_HEIGHT
    ).group_by(
      ExperimentDb.ImageOnlyShader.shader_id
    ).having(
      pw.fn.MAX(ExperimentDb.ImageOnlyExperiment.errors) == 0,
      pw.fn.SUM(ExperimentDb.ImageOnlyExperiment.trace.is_null(True)) == 0
    )
])

In [4]:
print(f"Number of shaders in Shadertoy Offline Database: {len(all_shaders)}")
print(f"Number of ImageOnly shaders: {len(imageonly_shaders)}")
print(f"Number of environments: {len(all_environments)}")

print(f"Number of measurable shaders: {len(measurable_shaders)}")
print(f"Number of all measurable shaders: {len(all_measurable_shaders)}")
print(f"Number of measurable and traceable shaders: {len(measurable_and_traceable_shaders)}")
print(f"Number of all measurable and traceable shaders: {len(all_measurable_and_traceable_shaders)}")
print(f"Number of all measurable and traceable canonical experimented shaders: {len(all_measurable_and_traceable_canonical_shaders)}")

Number of shaders in Shadertoy Offline Database: 0
Number of ImageOnly shaders: 0
Number of environments: 1
Number of measurable shaders: 13882
Number of all measurable shaders: 13882
Number of measurable and traceable shaders: 13868
Number of all measurable and traceable shaders: 4
Number of all measurable and traceable canonical experimented shaders: 13868


In [5]:
measurable_shaders - measurable_and_traceable_shaders

{'3tdGRj',
 '4llSzS',
 '7tlyDs',
 'Ds23Wd',
 'Nd33Dr',
 'NdfXR7',
 'Wd2yWt',
 'XdffR2',
 'XtXBzs',
 'dlc3Wl',
 'ld33RX',
 'ld3cWj',
 'msc3W2',
 'tddXzl'}

### shaderToExperiment

In [6]:
environment_selected = all_environments[0]
print(f"Environment selected: {environment_selected.node} - {environment_selected.cpu} - {environment_selected.gpu} - {environment_selected.gpu_driver}")

Environment selected: libreliu-GCL-Arch -  Intel(R) Core(TM) i7-10700K CPU @ 3.80GHz - NVIDIA GeForce RTX 3060 - NVIDIA 535.113.01


In [7]:
# Select experiments where shaderIDs match the shaderID provided, and
# return the first if have multiple record
shader_to_experiment = {}
query = ExperimentDb.ImageOnlyExperiment.select(
      ExperimentDb.ImageOnlyExperiment
    ).where(
      # Canonical condition
      ExperimentDb.ImageOnlyExperiment.num_cycles == ExperimentDb.CANONICAL_NUM_CYCLES,
      ExperimentDb.ImageOnlyExperiment.num_trials == ExperimentDb.CANONICAL_NUM_TRIALS,
      ExperimentDb.ImageOnlyExperiment.width == ExperimentDb.CANONICAL_WIDTH,
      ExperimentDb.ImageOnlyExperiment.height == ExperimentDb.CANONICAL_HEIGHT,
      # Inside all_measurable_and_traceable_canonical_shaders
      ExperimentDb.ImageOnlyExperiment.environment == environment_selected
    ).order_by(
      ExperimentDb.ImageOnlyExperiment.shader_shadertoy_id
    )

# To debug the SQL command use this
# print(query)
  
for expr in query:
  shader_id = expr.shader_shadertoy_id
  if shader_id not in all_measurable_and_traceable_canonical_shaders:
    continue

  if shader_id not in shader_to_experiment:
    shader_to_experiment[shader_id] = expr

print(f"Number of experiment selected: {len(shader_to_experiment)}")


Number of experiment selected: 13868
